In [1]:
# import libraries
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
from imdb import IMDb

In [30]:
# read the final aggregated movie data with additional information 
movie_data = pd.read_csv("../data/with_additional_data/military_hollywood_with_additional_data.csv")

In [41]:
reviews_all = []
ia = IMDb()
for index, row in movie_data.iterrows():
    movie_id = str(row.IMDB_ID)[2:]
    m_id = str(row.IMDB_ID)
    try:
        movie = ia.get_movie(movie_id, ['reviews'])
        movie['reviews'] = [dict(item, **{'idmb_id': m_id}) for item in movie['reviews']]
    except:
        continue
    reviews_all.append(movie['reviews'])

2021-04-24 22:50:47,459 CRITICAL [imdbpy] /opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/_exceptions.py:34: IMDbParserError exception raised; args: ('invalid movieID "ver Made": invalid literal for int() with base 10: \'ver Made\'',); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/parser/http/__init__.py", line 299, in _normalize_movieID
    return '%07d' % int(movieID)
ValueError: invalid literal for int() with base 10: 'ver Made'
2021-04-24 22:50:48,353 CRITICAL [imdbpy] /opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/_exceptions.py:34: IMDbParserError exception raised; args: ('invalid movieID "ver Made": invalid literal for int() with base 10: \'ver Made\'',); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/parser/http/__init__.py", line 299, in _normalize_movieID
    return '%07d' % int(movieID)
ValueError: invalid literal for int() with 

2021-04-24 22:53:20,290 CRITICAL [imdbpy] /opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/_exceptions.py:34: IMDbParserError exception raised; args: ('invalid movieID "n": invalid literal for int() with base 10: \'n\'',); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/parser/http/__init__.py", line 299, in _normalize_movieID
    return '%07d' % int(movieID)
ValueError: invalid literal for int() with base 10: 'n'
2021-04-24 22:53:23,333 CRITICAL [imdbpy] /opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/_exceptions.py:34: IMDbParserError exception raised; args: ('invalid movieID "n": invalid literal for int() with base 10: \'n\'',); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/parser/http/__init__.py", line 299, in _normalize_movieID
    return '%07d' % int(movieID)
ValueError: invalid literal for int() with base 10: 'n'
2021-04-24 22:53:33,79

2021-04-24 22:58:01,325 CRITICAL [imdbpy] /opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/_exceptions.py:34: IMDbParserError exception raised; args: ('invalid movieID "n": invalid literal for int() with base 10: \'n\'',); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/parser/http/__init__.py", line 299, in _normalize_movieID
    return '%07d' % int(movieID)
ValueError: invalid literal for int() with base 10: 'n'
2021-04-24 22:58:01,327 CRITICAL [imdbpy] /opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/_exceptions.py:34: IMDbParserError exception raised; args: ('invalid movieID "n": invalid literal for int() with base 10: \'n\'',); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/parser/http/__init__.py", line 299, in _normalize_movieID
    return '%07d' % int(movieID)
ValueError: invalid literal for int() with base 10: 'n'
2021-04-24 22:58:12,29

2021-04-24 23:00:22,207 CRITICAL [imdbpy] /opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/_exceptions.py:34: IMDbParserError exception raised; args: ('invalid movieID "n": invalid literal for int() with base 10: \'n\'',); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/parser/http/__init__.py", line 299, in _normalize_movieID
    return '%07d' % int(movieID)
ValueError: invalid literal for int() with base 10: 'n'
2021-04-24 23:00:32,212 CRITICAL [imdbpy] /opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/_exceptions.py:34: IMDbParserError exception raised; args: ('invalid movieID "n": invalid literal for int() with base 10: \'n\'',); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/parser/http/__init__.py", line 299, in _normalize_movieID
    return '%07d' % int(movieID)
ValueError: invalid literal for int() with base 10: 'n'
2021-04-24 23:00:32,92

2021-04-24 23:01:25,440 CRITICAL [imdbpy] /opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/_exceptions.py:34: IMDbParserError exception raised; args: ('invalid movieID "ver Made": invalid literal for int() with base 10: \'ver Made\'',); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/parser/http/__init__.py", line 299, in _normalize_movieID
    return '%07d' % int(movieID)
ValueError: invalid literal for int() with base 10: 'ver Made'
2021-04-24 23:01:25,440 CRITICAL [imdbpy] /opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/_exceptions.py:34: IMDbParserError exception raised; args: ('invalid movieID "ver Made": invalid literal for int() with base 10: \'ver Made\'',); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ada1/lib/python3.7/site-packages/imdb/parser/http/__init__.py", line 299, in _normalize_movieID
    return '%07d' % int(movieID)
ValueError: invalid literal for int() with 

In [61]:
len(reviews_all)

681

In [51]:
list_df_reviews = [pd.DataFrame(reviews_all[i]) for i in range(len(reviews_all))]

In [56]:
df_reviews = pd.concat(list_df_reviews)
df_reviews = df_reviews.reset_index(drop=True)

In [57]:
df_reviews.shape

(11818, 8)

In [59]:
df_reviews.head()

,content,helpful,title,author,date,rating,not_helpful,idmb_id
0,I accidently came across this movie while surf...,0,,ur0516638,24 June 2000,NaN,0,tt0133231
1,"This is a tearjerker, and knowing that this is...",0,,ur14569527,14 April 2007,1,0,tt0133231
2,This is one of those movies that will probably...,0,,ur2105110,20 December 2002,1,0,tt0133231
3,This is one of my favorite movies in the past ...,0,,ur1294947,4 August 2001,1,0,tt0133231
4,"The movie is pretty good, and relatively accur...",0,,ur97163633,1 January 2019,NaN,0,tt0133231
